In [1]:
from sents_dataset import LabeledDataset

In [2]:
labeled = LabeledDataset()

In [3]:
labeled.label_df()

Source  : habr
Text    : Понятно, что представление оценки в текущем виде не позволит достаточно точно определить людей со схожими интересами.

invalid command
valid commands: Y, N, S, EXIT
exit
